In [1]:
function travis(concurrent_jobs, macos)
    if concurrent_jobs ∉ (1, 2, 5)
        error("Number of concurrent Travis jobs must be in 1, 2, or 5")
    end

    total = 0.0
    total += if concurrent_jobs == 1
        69
    elseif concurrent_jobs == 2
        129
    elseif concurrent_jobs == 5
        249
    end

    # 15$ buys 25k credits, 1 macOS minute = 50 credits
    total += 15 * cld(macos, 500)
    return total
end

function github(plan, linux, macos, windows)
    if plan ∉ (:free, :pro, :team, :enterprise)
        error("GitHub Actions plan must be :free, :pro, :team or :enterprise")
    end
    
    if plan === :free
        free_minutes = 2000
        total = 0.0
    elseif plan in (:pro, :team)
        free_minutes = 3000
        # We can have a Team plan for free
        total = 0.0
    elseif plan === :enterprise
        free_minutes = 50_000
        # They offer a University license: https://enterprise.github.com/faq#faq-23,
        # but I have no idea what would be the price, let's guess
        total = 100.0
    end

    total_minutes = linux + macos + windows
    if iszero(total_minutes)
        # Fast-track: no extra minute actually used,
        # return the current total
        return round(total; digits=2)
    end

    # Some guesswork: remove from the total minutes for each OS
    # a time proportional to the free/total time ratio
    linux, macos, windows = (linux, macos, windows) .* max(0, total_minutes - free_minutes) ./ total_minutes

    total += linux * 0.008
    total += macos * 0.08
    total += windows * 0.016
    return round(total; digits=2)
end

struct Summary
    linux::Float64
    macos::Float64
    windows::Float64
    travis_jobs::Int
    github_plan::Symbol
end

function Base.show(io::IO, s::Summary)
    println(io, "Travis: ", travis(s.travis_jobs, s.macos), "\$/month")
    println(io, "GitHub: ", github(s.github_plan, s.linux, s.macos, s.windows), "\$/month")
end

In [2]:
using Interact

travis_wdg = radiobuttons(Observable(Dict("1 Travis job" => 1, "2 Travis jobs" => 2, "5 Travis jobs" => 5)); 
    value=5)
github_wdg = radiobuttons(Observable(Dict(
            "GitHub Actions Free" => :free,
            "GitHub Actions Pro" => :pro, 
            "GitHub Actions Team" => :team,
            "GitHub Actions Enterprise" => :enterprise)); value=:team)

minutes = 0.0:100.0:50000.0

@manipulate for linux_minutes in minutes,
    macos_minutes in minutes,
    windows_minutes in minutes,
    travis_jobs in travis_wdg,
    github_plan in github_wdg

    Summary(linux_minutes, macos_minutes, windows_minutes, travis_jobs, github_plan)
end

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-2959689523694677764\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-14715963753565641140\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-2959689523694677764\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["linux_minutes"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 501,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
251, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/mose/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/mose/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/mose/.julia/packages/InteractBase/sOe2Z/src/../assets/all.js"), Asset("css", nothing, "/home/mose/.julia/packages/InteractBase/sOe2Z/src/../assets/style.css"), Asset("css", nothing, "/home/mose/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f60a93b7d00, Task (runnable) @0x00007f60a93b7d00), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"100.0\",\"200.0\",\"300.0\",\"400.0\",\"500.0\",\"600.0\",\"700.0\",\"800.0\",\"900.0\",\"1000.0\",\"1100.0\",\"1200.0\",\"1300.0\",\"1400.0\",\"1500.0\",\"1600.0\",\"1700.0\",\"1800.0\",\"1900.0\",\"2000.0\",\"2100.0\",\"2200.0\",\"230